In [ ]:
# %conda env export > environment.yaml
%conda env create -f environment.yaml

In [ ]:
# %pip install mne pandas numpy scikit-learn mne-icalabel
%pip install ipywidgets

# TMS pipeline

This notebook contains the full pipeline of the individual scripts. The pipeline goes as follows:

- EDA
- Prepocessing
- Model training
- Validation


In [1]:
# Imports
import os
import pandas as pd
from ipywidgets import *
import numpy as np
import mne
from mne.preprocessing import ICA

# import torch
from mne_icalabel import label_components

import matplotlib.pyplot as plt
from mne.preprocessing import ICA
import scipy

import preprocessing
import utils

# Specify graph rendering method
# %matplotlib widget
plt.switch_backend("TkAgg")

## Data loading

Currently, only one session from one patient gets used for testing purposes. File names are as follows: "**TMS-EEG-H_02_S1b_X_Y.Z**", where:

- X = **rsEEG** (resting state EEG) or **spTEP** (single pulse TMS Evoked Potential)
- Y = **pre** or **post** (before or after the rTMS procedure)
- Z = **vhdr**, **vmrk**, **eeg** or **mat** (files for the BrainVision format, and a MATLAB file)


In [2]:
# Currently for 1 patient, will be generalized into a pipeline for all patients

DATASET_PATH = "./dataset"
FILENAME_TEMPLATE = "TMS-EEG-H_02_S1b_{}_{}.vhdr"

# rsEEG_pre_raw = mne.io.read_raw_brainvision(os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("rsEEG", "pre")), preload=True)
spTEP_pre_raw = mne.io.read_raw_brainvision(
    os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("spTEP", "pre")), preload=True
)
spTEP_pre_raw.drop_channels(["HEOG", "VEOG"])

# rsEEG_post_raw = mne.io.read_raw_brainvision(os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("rsEEG", "post")), preload=True)
# spTEP_post_raw = mne.io.read_raw_brainvision(os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("spTEP", "post")), preload=True)
# spTEP_post_raw.drop_channels(['HEOG', 'VEOG'])

Extracting parameters from ./dataset\TMS-EEG-H_02_S1b_spTEP_pre.vhdr...
Setting channel info structure...
Reading 0 ... 2696199  =      0.000 ...   539.240 secs...


Measurement date,"August 23, 2017 15:29:56 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,67 points
Good channels,62 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,5000.00 Hz
Highpass,0.00 Hz
Lowpass,1000.00 Hz


## EDA

### EEG Visualization

Visualizing the EEG graphs and electrodes for all the files gives a first impression of the data, and possible immediate preprocessing changes that can take place.

spTEP_pre: TP9 is bad channel

Perfect overlap on the Fp2 and VEOG channels, likely because the Fp2 was also used as a reference for VEOG. This overlap does result in errors, so it's better to remove the VEOG channel and keep this reference in mind.


In [ ]:
spTEP_pre_raw.plot(start=60, duration=10, n_channels=20, scalings={"eeg": 50e-6})

In [ ]:
utils.plot_single_response(spTEP_pre_raw, channel="Pz", tmin=-0.05, tmax=0.05)

In [ ]:
print(spTEP_pre_raw.info)

In [ ]:
utils.plot_average_response(spTEP_pre_raw, tmin=-0.01, tmax=0.05)

In [ ]:
# Clearly indicates where the coils were placed
spTEP_pre_raw.compute_psd().plot_topomap()

## Preprocessing


In [4]:
# Preprocessing with interpolation
spTEP_pre_raw.info["bads"] = ["TP9"]
spTEP_pre_raw.interpolate_bads(reset_bads=True)

spTEP_pre = spTEP_pre_raw.copy()
spTEP_pre = preprocessing.preprocess_spTEP(spTEP_pre)

# spTEP_post = spTEP_post_raw.copy()
# spTEP_post = preprocessing.preprocess(spTEP_post)

Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.0 mm
Computing interpolation matrix from 61 sensor positions
Interpolating 1 sensors
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
151 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 151 events and 2001 original time points ...
139 bad epochs dropped
Fitting ICA to data using 62 channels (please be patient, this may take a while)
Selecting by number: 20 components


d:\Coding\tms-research\preprocessing.py:79: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(epochs)
d:\Coding\tms-research\preprocessing.py:79: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)


Fitting ICA took 1.7s.
Effective window size : 2.048 (s)
Applying ICA to Raw instance
    Transforming to ICA space (20 components)
    Zeroing out 1 ICA component
    Projecting back using 62 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 90 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 90.00 Hz
- Upper transition bandwidth: 22.50 Hz (-6 dB cutoff frequency: 101.25 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Not setting metadata
151 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 151 events and 2001 original time points ...
139 bad epochs dropped
Fitting ICA to data using 62 channels (please be patient, this may take a while)
Selecting by number: 20 components


d:\Coding\tms-research\preprocessing.py:79: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)


Fitting ICA took 0.5s.
Effective window size : 2.048 (s)
Applying ICA to Raw instance
    Transforming to ICA space (20 components)
    Zeroing out 0 ICA components
    Projecting back using 62 PCA components
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


In [4]:
utils.plot_average_response(spTEP_pre_raw, tmin=-0.005, tmax=0.2)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
151 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 151 events and 1026 original time points ...
1 bad epochs dropped


In [5]:
# Full pipeline
utils.plot_average_response(spTEP_pre, tmin=-0.05, tmax=0.2)
# utils.plot_average_response(spTEP_post, tmin=-0.05, tmax=0.2)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
151 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 151 events and 251 original time points ...
1 bad epochs dropped


In [ ]:
FILENAME_TEMPLATE = "TMS-EEG-H_{:02d}_S{}_{}_{}"
filename = FILENAME_TEMPLATE.format(2, "1b", "spTEP", "pre")
spTEP_pre.save(os.path.join(".", "cleaned", filename + ".fif"), overwrite=True)

In [ ]:
# Apply preprocessing to all available data and save for later use

DATASET_PATH_FULL = "../neuroaa/datasets/raw/uz_gent"
FILENAME_TEMPLATE = "TMS-EEG-H_{:02d}_S{}_{}_{}"

for patient in range(19):
    for session in range(4):
        for type in ["spTEP", "rsEEG"]:
            for trial in ["pre", "post"]:
                filename = FILENAME_TEMPLATE.format(patient, session, type, trial)
                eeg_raw = mne.io.read_raw_brainvision(
                    os.path.join(DATASET_PATH, filename + ".vhdr"), preload=True
                )
                eeg_clean = eeg_raw.copy()
                eeg_clean = preprocessing.preprocess(eeg_clean)
                eeg_clean.save(
                    os.path.join(".", "cleaned_data", filename + ".fif"), overwrite=True
                )

## Feature extraction

Features are per pulse (spTEP) or epoch (rsEEG)


### Time domain


In [ ]:
# Peak amplitude, peak latency, and area under curve
events, event_dict = mne.events_from_annotations(spTEP_pre)
epochs = mne.Epochs(
    spTEP_pre,
    events,
    event_id=event_dict,
    tmin=-0.05,
    tmax=0.2,
    baseline=None,
    preload=True,
)

features = []

for epoch in epochs.get_data():
    peak_amplitude = np.max(epoch)
    peak_latency = np.argmax(epoch) / epochs.info["sfreq"]
    area_under_curve = np.mean(np.trapz(epoch))

    features.append([peak_amplitude, peak_latency, area_under_curve])

len(features)

In [ ]:
# Plot difference of features between pre and post

events, event_dict = mne.events_from_annotations(spTEP_pre)
epochs = mne.Epochs(
    spTEP_pre,
    events,
    event_id=event_dict,
    tmin=-0.05,
    tmax=0.2,
    baseline=None,
    preload=True,
)
data = epochs.get_data()
average_response = epochs.average().data

events_post, event_dict_post = mne.events_from_annotations(spTEP_post)
epochs_post = mne.Epochs(
    spTEP_post,
    events_post,
    event_id=event_dict_post,
    tmin=-0.05,
    tmax=0.2,
    baseline=None,
    preload=True,
)
data_post = epochs_post.get_data()
average_response_post = epochs_post.average().data

average_max = average_response.max(axis=1)
mean = data.mean(axis=(0, 2))
std_dev = data.std(axis=(0, 2))
skewness = scipy.stats.skew(data, axis=(0, 2))
kurtosis = scipy.stats.kurtosis(data, axis=(0, 2))

average_max_post = average_response_post.max(axis=1)
mean_post = data_post.mean(axis=(0, 2))
std_dev_post = data_post.std(axis=(0, 2))
skewness_post = scipy.stats.skew(data_post, axis=(0, 2))
kurtosis_post = scipy.stats.kurtosis(data_post, axis=(0, 2))

fig, axs = plt.subplots(3, 2)
axs[0, 0].scatter(mean, mean_post)
axs[0, 0].set_xlabel("Mean (pre)")
axs[0, 0].set_ylabel("Mean (post)")

axs[0, 1].scatter(std_dev, std_dev_post)
axs[0, 1].set_xlabel("Standard Deviation (pre)")
axs[0, 1].set_ylabel("Standard Deviation (post)")

axs[1, 0].scatter(skewness, skewness_post)
axs[1, 0].set_xlabel("Skewness (pre)")
axs[1, 0].set_ylabel("Skewness (post)")

axs[1, 1].scatter(kurtosis, kurtosis_post)
axs[1, 1].set_xlabel("Kurtosis (pre)")
axs[1, 1].set_ylabel("Kurtosis (post)")

axs[2, 0].scatter(average_max, average_max_post)
axs[2, 0].set_xlabel("Max Average Response (pre)")
axs[2, 0].set_ylabel("Max Average Response (post)")

# Remove the unused subplot
fig.delaxes(axs[2, 1])

plt.tight_layout()
plt.show()

### Frequency domain


In [ ]:
bands = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta": (12, 30),
    "gamma": (30, 50),
}

In [ ]:
# Average power per frequency band
psd = spTEP_pre.compute_psd(fmin=1.0, fmax=100.0)
psd.plot(average=True)

freqs = psd.freqs

# Initialize a dictionary to hold the average power for each band
avg_power = {}

for band, (fmin, fmax) in bands.items():
    band_indices = np.where((freqs >= fmin) & (freqs <= fmax))[0]
    band_psd = psd.get_data()[:, band_indices]
    avg_power[band] = np.mean(band_psd)

print(avg_power)

In [ ]:
# Difference in average power
psd_pre = epochs.compute_psd(fmin=1.0, fmax=100.0)
psd_post = epochs_post.compute_psd(fmin=1.0, fmax=100.0)

psd_pre_avg = psd_pre.average()
psd_post_avg = psd_post.average()

psd_diff = psd_post_avg.get_data() - psd_pre_avg.get_data()

fig, ax = plt.subplots()

# Plot the difference PSD
ax.plot(psd_pre.freqs, psd_diff[0], label="Post - Pre")

# Add a legend
ax.legend()

# Show the plot
plt.show()

### Time-frequency domain


In [ ]:
# Compute the wavelet transform of the data
events, event_dict = mne.events_from_annotations(spTEP_pre)
epochs = mne.Epochs(
    spTEP_pre,
    events,
    event_id=event_dict,
    tmin=0,
    tmax=10,
    baseline=None,
    preload=True,
)

frequencies = np.arange(1, 100, 2)
n_cycles = frequencies / 2

interpolated_start = -0.002
interpolated_end = 0.005
interpolated_start_idx = int(interpolated_start * epochs.info["sfreq"])
interpolated_end_idx = int(interpolated_end * epochs.info["sfreq"])

epochs_data = epochs.get_data()[:, :, :interpolated_start_idx]
epochs_data = np.concatenate(
    [epochs_data, epochs.get_data()[:, :, interpolated_end_idx:]], axis=2
)
epochs_cleaned = mne.EpochsArray(epochs_data, epochs.info, events=epochs.events)


wavelets = mne.time_frequency.tfr_morlet(
    epochs_cleaned, freqs=frequencies, n_cycles=5, n_jobs=-1
)

In [ ]:
# Compute the wavelet transform of the data
events, event_dict = mne.events_from_annotations(spTEP_pre)
n_trials = 20
epochs = mne.Epochs(
    spTEP_pre,
    events,
    event_id=event_dict,
    tmin=0,
    tmax=n_trials,
    baseline=None,
    preload=True,
)

frequencies = np.arange(1, 100, 2)

wavelets = mne.time_frequency.tfr_morlet(
    epochs, freqs=frequencies, n_cycles=n_trials / 2, n_jobs=-1
)

In [ ]:
# INDIVIDUAL PLOTS

power = wavelets[0].data
avg_power = np.mean(power, axis=0)

fig, ax = plt.subplots()

# Create the heatmap
cax = ax.imshow(avg_power, aspect="auto", cmap="hot", origin="lower")

# Add a colorbar
fig.colorbar(cax)

# Set the labels for the x and y axes and the title
ax.set_xlabel("Time")
ax.set_ylabel("Frequency")
ax.set_title("Average Power")

# Show the plot
plt.show()

# GROUPED PLOTS
freqs = wavelets[0].freqs

n_rows = len(bands)
n_cols = 1

# Create a figure with multiple subplots
fig, axs = plt.subplots(n_rows, n_cols, figsize=(10, 20))

for ax, (band, (fmin, fmax)) in zip(axs, bands.items()):
    # Find the indices that correspond to this frequency band
    band_indices = np.where((freqs >= fmin) & (freqs <= fmax))[0]

    # Slice the power data to include only these frequencies
    band_power = power[:, band_indices, :]

    # Compute the average power across electrodes
    avg_power = np.mean(band_power, axis=0)

    # Create the heatmap
    cax = ax.imshow(avg_power, aspect="auto", cmap="hot", origin="lower")

    ax.set_xlim([None, 10000])

    # Add a colorbar
    fig.colorbar(cax, ax=ax)

    # Set the labels for the x and y axes and the title
    ax.set_xlabel("Time")
    ax.set_ylabel("Frequency")
    ax.set_title(f"Average Power ({band} band)")

# Show the plot
plt.tight_layout()
plt.show()

## Clustering


In [8]:
clone = spTEP_pre_raw.copy()

sampling_rate = clone.info["sfreq"]
events, event_dict = mne.events_from_annotations(clone)
tms_indices = [event[0] for event in events if event[2] == 1]

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']


In [4]:
def remove_EOG(eeg_data):
    if "HEOG" in eeg_data.ch_names:
        eeg_data.drop_channels(["HEOG"])
    if "VEOG" in eeg_data.ch_names:
        eeg_data.drop_channels(["VEOG"])


def remove_range(
    eeg_data_raw, tms_indices, start, end, sampling_rate, interpolate=True
):
    """Replace all the data in the range around the TMS pulse with 0

    Args:
        eeg_data (RawBrainVision): Raw EEG data
        tms_indices (array): Timestamps of the TMS pulses
        start (int): Time before the TMS pulse to start removing data
        end (int): Time after the TMS pulse to stop removing data
        sampling_rate (int): Rate at which the EEG was sampled

    Returns:
        np.array: Filtered data
    """
    eeg_data = eeg_data_raw.get_data()
    num_electrodes = eeg_data.shape[0]
    if interpolate:
        for tms_index in tms_indices:
            start_index, end_index = calculate_range_indices(
                tms_index, start, end, sampling_rate
            )
            for i in range(num_electrodes):
                x = [start_index - 1, end_index + 1]
                y = [eeg_data[i, start_index - 1], eeg_data[i, end_index + 1]]
                x_new = np.arange(start_index, end_index + 1)
                eeg_data[i, start_index : end_index + 1] = np.interp(x_new, x, y)

    else:
        for tms_index in tms_indices:
            start_index, end_index = calculate_range_indices(
                tms_index, start, end, sampling_rate
            )
            for i in range(num_electrodes):
                eeg_data[i, start_index : end_index + 1] = 0

    eeg_data_raw._data = eeg_data


def downsample(eeg_data, sample_rate=1000):
    eeg_data.resample(sample_rate, npad="auto")


def calculate_range_indices(tms_index, start, end, sampling_rate):
    samples_before = int(start * sampling_rate)
    samples_after = int(end * sampling_rate)

    start_index = max(0, tms_index - samples_before)
    end_index = tms_index + samples_after

    return start_index, end_index


def ICA_filter(eeg_data, events, event_dict, baseline=(-1, -0.002)):
    epochs = mne.Epochs(
        eeg_data,
        events,
        event_id=event_dict,
        tmin=-1,
        tmax=1,
        baseline=baseline,
        preload=True,
    )
    ica = mne.preprocessing.ICA(n_components=20, random_state=42)
    ica.fit(epochs)
    muscle_idx_auto, scores = ica.find_bads_muscle(eeg_data)
    ica.exclude = muscle_idx_auto
    eeg_data = ica.apply(eeg_data)


def bandpass(eeg_data, low_freq=1, high_freq=90):
    eeg_data.filter(low_freq, high_freq)


def notch(eeg_data, freqs=[50]):
    eeg_data.notch_filter(freqs)


def rereference(eeg_data):
    # mne.set_eeg_reference(eeg_data, ref_channels=['Cz'], projection=False)
    mne.set_eeg_reference(eeg_data, ref_channels="average")


def preprocess_spTEP(eeg_data):
    """Filters the EEG data and removes artifacts. Changes the original data in place.

    POSSIBLE IMPROVEMENT: demeaning instead of baseline correction

    Args:
        eeg_data (RawBrainVision): RawBrainVision object containing the EEG data

    Returns:
        np.array: numpy array of the preprocessed EEG data
    """
    sampling_rate = eeg_data.info["sfreq"]
    events, event_dict = mne.events_from_annotations(eeg_data)
    tms_indices = [event[0] for event in events if event[2] == 1]

    remove_EOG(eeg_data)
    remove_range(eeg_data, tms_indices, 0.002, 0.005, sampling_rate)
    downsample(eeg_data)
    ICA_filter(
        eeg_data, events, event_dict
    )  # TODO: apply ICA filter using a rule, not automatic detection
    bandpass(eeg_data)
    notch(eeg_data)
    ICA_filter(eeg_data, events, event_dict)
    rereference(eeg_data)
    return eeg_data

In [9]:
clone = preprocess_spTEP(clone)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
151 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 151 events and 2001 original time points ...
139 bad epochs dropped
Fitting ICA to data using 62 channels (please be patient, this may take a while)
Selecting by number: 20 components


C:\Users\Tomas\AppData\Local\Temp\ipykernel_22716\1530077958.py:72: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(epochs)
C:\Users\Tomas\AppData\Local\Temp\ipykernel_22716\1530077958.py:72: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)


Fitting ICA took 0.7s.
Effective window size : 2.048 (s)
Applying ICA to Raw instance
    Transforming to ICA space (20 components)
    Zeroing out 1 ICA component
    Projecting back using 62 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 90 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 90.00 Hz
- Upper transition bandwidth: 22.50 Hz (-6 dB cutoff frequency: 101.25 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Not setting metadata
151 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 151 events and 2001 original time points ...
139 bad epochs dropped
Fitting ICA to data using 62 channels (please be patient, this may take a while)
Selecting by number: 20 components


C:\Users\Tomas\AppData\Local\Temp\ipykernel_22716\1530077958.py:72: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)


Fitting ICA took 0.6s.
Effective window size : 2.048 (s)
Applying ICA to Raw instance
    Transforming to ICA space (20 components)
    Zeroing out 0 ICA components
    Projecting back using 62 PCA components
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


```py
remove_EOG(eeg_data)
remove_range(eeg_data, tms_indices, 0.002, 0.005, sampling_rate)
downsample(eeg_data)
ICA_filter(
    eeg_data, events, event_dict
)  # TODO: apply ICA filter using a rule, not automatic detection
bandpass(eeg_data)
notch(eeg_data)
ICA_filter(eeg_data, events, event_dict)
rereference(eeg_data)
```


In [13]:
# remove_EOG(clone)
# remove_range(clone, tms_indices, 0.002, 0.005, sampling_rate)
# downsample(clone)
# ICA_filter(clone, events, event_dict)
# bandpass(clone)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 90 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 90.00 Hz
- Upper transition bandwidth: 22.50 Hz (-6 dB cutoff frequency: 101.25 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


In [15]:
utils.plot_average_response(clone, tmin=-0.05, tmax=0.2)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
151 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 151 events and 251 original time points ...
1 bad epochs dropped
